# LINEARISATION

In [42]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib, os, fiona
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime
from collections import Counter
from sqlalchemy import Table, Column, Integer, String, Float,MetaData
from geoalchemy2 import Geometry
from shapely.wkt import dumps

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## CD17
Réutiliser les comptages importés grace au Notebooj Import_trafics (chap cd17) et les données de troncons elemnetaires issues du notebook agrege_troncon pour déterminer les nouveaux id_comptag 

In [2]:
#import des données de troncon elem (à modifier par imporft depuis Bdd quand le transfert vers Bdd en sortie d'agreg_troncon sera calé)
gdf_traf=gp.read_file(r'D:\temp\otv\test_linearisation\df_lignes_fin_tot.shp')

In [3]:
#import des points de comptages depuis la Bdd
with ct.ConnexionBdd('gti_otv') as c :
    rqt="select * from comptage.na_2010_2017_p where dep='17' and geom is not null"
    gdf_pt_cpt = gp.read_postgis(rqt, c.connexionPsy)
#mise en forme attributs
def convert_tmja(df) : 
    for annee in [a for a in range(2017, 2009,-1)]+['autre'] :
        attr='tmja_'+str(annee)
        if ~np.isnan(df[attr]) : 
            return (df[attr], annee)
df_adj=gdf_pt_cpt.apply(lambda x : convert_tmja(x), axis=1,result_type='expand')
df_adj.columns=['tmja_recent','annee_tmja_recent']
gdf_pt_cpt=gdf_pt_cpt.merge(df_adj, left_index=True, right_index=True)#[['id_comptag','geom','type_poste','tmja_recent','annee_tmja_recent' ]]

In [4]:
#filtre des points ponctuels en été
pt_pctuels_ete=gdf_pt_cpt.loc[gdf_pt_cpt['type_poste']=='ponctuel'].copy()
def verif_ete(tmja_2015, obs_2015, tmja_2016, obs_2016) :
    if tmja_2016 > 0 :
        return any([a in ['July','August']  for a in [pd.to_datetime(obs_2016.split(',')[1].split('-')[i], format='%d/%m/%Y').month_name()for i in [0,1]]])
    elif tmja_2015 > 0 :
        return any([a in ['July','August']  for a in [pd.to_datetime(obs_2015.split(',')[1].split('-')[i], format='%d/%m/%Y').month_name()for i in [0,1]]])
    else : return False
pt_pctuels_ete['ete']=pt_pctuels_ete.apply(lambda x : verif_ete(x['tmja_2015'],x['obs_2015'], x['tmja_2016'], x['obs_2016']), axis=1)
pt_pctuels_ete=pt_pctuels_ete.loc[~pt_pctuels_ete['ete']].copy()
gdf_pt_cpt=gdf_pt_cpt.loc[gdf_pt_cpt.index.isin(pt_pctuels_ete.index.tolist()+gdf_pt_cpt.loc[gdf_pt_cpt['type_poste']!='ponctuel'].index.tolist())].copy()

In [5]:
#croisement point et lignes (sur labase d'un buffer de 0.5m)
gdf_pt_cpt.geometry=gdf_pt_cpt.buffer(0.5)
gdf_traf_pt=gp.sjoin(gdf_traf,gdf_pt_cpt,how="left", op='intersects' )[['id_ign','tmja', 'importance',
            'id_comptag_left', 'id_tronc_e', 'id_comptag_right','type_poste','tmja_recent','annee_tmja_recent','source','target', 'geometry']].rename(
columns={'id_comptag_left':'id_cpt_lin','id_comptag_right': 'id_cpt_pt','tmja':'tmja_lin'})

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [6]:
#recherch des tronc_elem avec plusieurs points
gdf_traf_pt_notna=gdf_traf_pt.loc[~gdf_traf_pt['id_cpt_pt'].isna()].copy()
tronc_elem_pt_cpt=gdf_traf_pt_notna.sort_index().groupby('id_tronc_e').agg({'id_cpt_pt' : lambda x : tuple(x),
                                       'type_poste' : lambda x : tuple(x),
                                        'tmja_recent' : lambda x : tuple(x),
                                        'annee_tmja_recent' : lambda x : tuple(x)})
trc_elem_multipt=tronc_elem_pt_cpt.loc[tronc_elem_pt_cpt.apply(lambda x : len(x['id_cpt_pt'])>1,axis=1)].copy()

In [7]:
#mise en forme des trocnon supportant plusieurs point : 
    #gestion des trocnon avec plusieurs points differntes (on garde le ponctuele de trafic max de l'annee la plus recente si que des ponctuels, sinon le tournant ou permanent)
    #verif de la validite des points en comprant les valeurs de trafic entre elle: si un des trafics est sup 1000 et que la variation est sup 30 %, on n'affecte pas
def cpt_final (type_poste,id_comptag_right,tmja_recent,annee_tmja_recent ): 
    tmja_recent=np.array(tmja_recent) 
    id_comptag_right=np.array(id_comptag_right)
    mask_tmja = (tmja_recent==max(tmja_recent))
    if all(np.unique(type_poste)=='ponctuel') :
        mask_annee=np.array(annee_tmja_recent)==max(annee_tmja_recent)
        if all(mask_annee):#il n'y a qu'une seule annee
            if all(mask_tmja) : 
                return id_comptag_right[0]
            else : 
                return id_comptag_right[mask_tmja][0]
        else : #il y a pluseurs annee dc on ne garde que le max de la bonne annee
            return id_comptag_right[mask_tmja]
    else :
        mask_type_poste=np.isin(np.array(type_poste),['permanent', 'tournant'])#on trouve le(s) poste(s) non ponctuel
        return id_comptag_right[mask_type_poste][0]
#verif de la validite des points en comprant les valeurs de trafic entre elle: si un des trafics est sup 1000 et que la variation est sup 30 %, on n'affecte pas
trc_elem_multipt['id_cpt_final']=trc_elem_multipt.apply(lambda x: cpt_final(
    x['type_poste'], x['id_cpt_pt'], x['tmja_recent'], x['annee_tmja_recent']), axis=1)
trc_elem_multipt['valide']=trc_elem_multipt.apply(lambda x : (False if abs(100-(min(x['tmja_recent'])/max(x['tmja_recent'])*100))>30 and
                                                                    max(x['tmja_recent'])>1000 and all(np.unique(x['type_poste'])=='ponctuel') else True), axis=1)
trc_elem_multipt=trc_elem_multipt.loc[trc_elem_multipt['valide']].copy()

In [8]:
#obtention de la df des id_comptage_pt par tronc_elem
tronc_elem_pt_cpt.update(trc_elem_multipt.drop('id_cpt_pt',axis=1).rename(columns={'id_cpt_final':'id_cpt_pt'})[['id_cpt_pt']])
tronc_elem_pt_cpt['id_cpt_pt']=tronc_elem_pt_cpt.apply(lambda x : x['id_cpt_pt'][0] if isinstance(x['id_cpt_pt'], tuple) else x['id_cpt_pt'],axis=1) #uniformisation du type de données en str

In [9]:
#transfert de l'id_cpt_pt final dans la df source
gdf_traf_pt_final=gdf_traf_pt[['id_ign','importance', 'id_cpt_lin','tmja_lin', 'id_tronc_e','geometry','source','target']].merge(tronc_elem_pt_cpt[['id_cpt_pt']], how='left',left_on='id_tronc_e', right_index=True)

In [10]:
#pour info, ligne ign non linéarisée qui le deviennent
gdf_creation_idcpt=gdf_traf_pt_final.loc[(gdf_traf_pt_final['id_cpt_lin'].isna()) & (~gdf_traf_pt_final['id_cpt_pt'].isna())]
#pour info, ligne ign linéarisée qui le sont toujours, mais peut avec un nouvel id_ign
gdf_variation_idcpt=gdf_traf_pt_final.loc[(~gdf_traf_pt_final['id_cpt_lin'].isna()) & (~gdf_traf_pt_final['id_cpt_pt'].isna()) & (gdf_traf_pt_final['id_cpt_pt']!=gdf_traf_pt_final['id_cpt_lin'])].sort_values('id_cpt_lin')

### mettre à jour les lignes des troncons qui appartiennent à des id_comptag qui supporte un nouvel id_cpt
exemple avec l'idcomptag '17-D734-19+620' pour le point de comptage nouveau '17-D734-28+170'
1. isoler les lignes qui sont relaticves à l'id_cpt_lin et trouver l'importance la plus représentée
1. trouver les lignes qui intersectent celles de l'idcompatg etudie dont l'importance est = ou sup
1. faire un graph avec les lignes de l'id_cpt_lin et celles qui interectent avec la bonne importance
1. trouver toute les lignes concernées par l'id_cpt_pt à partir du nouveau graph (plus tard quand on trouvera facilement les troncons elemenraires cela ne sera plus utile) comme ça on ne s'embarasse pas des pb de croisement avec des lignes de moindre importance

In [ ]:
#isoler les lignes qui sont relaticves à l'id_cpt_lin et trouver l'importance la plus représentée
for id_cpt_a_test in gdf_variation_idcpt.id_cpt_lin.unique() : 
    print(id_cpt_a_test)
    lgn_idcpt_lin=gdf_traf.loc[gdf_traf['id_comptag']==id_cpt_a_test].copy()
    importance=[k for k, v in Counter(lgn_idcpt_lin.importance.tolist()).items() if v==max( Counter(lgn_idcpt_lin.importance.tolist()).values())][0]

    #trouver les lignes qui intersectent celles de l'idcompatg etudie dont l'importance est = ou sup
        #list des source / target de l'idcomptage
    list_src_tgt=lgn_idcpt_lin.source.tolist()+lgn_idcpt_lin.target.tolist()
    df_lgn_interscts=gdf_traf.loc[((gdf_traf['source'].isin(list_src_tgt)) | (gdf_traf['target'].isin(list_src_tgt))) & (gdf_traf['importance'] <=importance) &
                                 (~gdf_traf['id_ign'].isin(lgn_idcpt_lin.id_ign.tolist()))]

    #faire un graph avec les lignes de l'id_cpt_lin et celles qui interectent avec la bonne importance
        #regrouper l'ensemble des lignes dans une df 
    ligne_pr_graph=gdf_traf.loc[gdf_traf['id_ign'].isin(df_lgn_interscts.id_ign.tolist()+lgn_idcpt_lin.id_ign.tolist())]
    ligne_pr_graph_transfert=ligne_pr_graph.copy()
    ligne_pr_graph_transfert['geom']=ligne_pr_graph_transfert.apply(lambda x : dumps(x.geometry), axis=1)
        #faire un graph à partir de ces lignes
    bdd='gti_otv'
    schema='public'
    table='graph_temp'
    table_vertex='graph_temp_vertices_pgr'
    with ct.ConnexionBdd(bdd) as c:
        metadata = MetaData(schema=schema)
        #supprimer table si elle existe
        rqt=f"drop table if exists {schema}.{table} ; drop table if exists {schema}.{table_vertex} "
        c.sqlAlchemyConn.execute(rqt)

        #creer nouvelle table
        graph = Table('graph_temp', metadata,
            Column('id', Integer, primary_key=True),
            Column('id_ign', String),
            Column('importance', String),
            Column('numero', String),
            Column('nature', String),
            Column('codevoie_d', String),
            Column('sens', String),
            Column('tmja', Integer),
            Column('source', Integer),
            Column('target', Integer),
            Column('long_km', Float),
            Column('geom',Geometry()))
        metadata.create_all(c.engine)
        ligne_pr_graph_transfert[['id','id_ign','importance','numero','tmja','source','target','long_km','geom']].to_sql(
            table,c.sqlAlchemyConn,schema=schema,if_exists='append', index=False )
        #creer le graph
        rqt_creation_graph_transfert=f"""update {schema}.{table} set geom=st_Multi(st_setsrid(geom,2154)), source=null, target=null ; 
                             select pgr_createTopology('{schema}.{table}', 0.001,'geom')"""
        c.sqlAlchemyConn.execute(rqt_creation_graph)
        rqt_anlyse_graph=f"SELECT pgr_analyzeGraph('{schema}.{table}', 0.001,\'geom\')"
        c.curs.execute(rqt_anlyse_graph)#je le fait avec psycopg2 car avec sql acchemy ça ne passe pas
        c.connexionPsy.commit()

    #trouver une ligne relative à chaque nouveau point
    liste_new_pt=gdf_variation_idcpt.loc[gdf_variation_idcpt['id_cpt_lin']==id_cpt_a_test].id_cpt_pt.unique()
    if len(liste_new_pt)==1 :
        liste_new_pt=liste_new_pt[0]
        dico_ligne_pt=gdf_variation_idcpt.loc[gdf_variation_idcpt['id_cpt_pt']==liste_new_pt].groupby('id_cpt_pt').agg({'id_ign': 'max',
                                            'id_tronc_e' : 'max'}).to_records()
    else : 
        dico_ligne_pt=gdf_variation_idcpt.loc[gdf_variation_idcpt['id_cpt_pt'].isin(liste_new_pt)].groupby('id_cpt_pt').agg({'id_ign': 'max',
                                            'id_tronc_e' : 'max'}).to_records()
    dico_ligne_pt={k[0] : {'ligne_base': k[1],'trc_elem':k[2],'lgn_trc_elem' : gdf_traf_pt.loc[gdf_traf_pt['id_tronc_e']==k[2]].id_ign.tolist(), 'priorite':2} for k in dico_ligne_pt}
    dico_ligne_pt['17-D734-19+620']={'ligne_base' : gdf_traf_pt.loc[(gdf_traf_pt['type_poste']=='permanent') & 
                                                                    (gdf_traf_pt['id_cpt_lin']=='17-D734-19+620')].id_ign.values[0],
                                     'trc_elem':gdf_traf_pt.loc[(gdf_traf_pt['type_poste']=='permanent') & 
                                                                    (gdf_traf_pt['id_cpt_lin']=='17-D734-19+620')].id_tronc_e.values[0], 
                                     'lgn_trc_elem' : gdf_traf_pt.loc[gdf_traf_pt['id_tronc_e']==gdf_traf_pt.loc[(gdf_traf_pt['type_poste']=='permanent') & 
                                                                    (gdf_traf_pt['id_cpt_lin']=='17-D734-19+620')].id_tronc_e.values[0]].id_ign.tolist(),
                                     'priorite':1}
    dico_ligne_pt

    #pour chaque ligne : on fait tourner la fonction de tronc elem à partir du graph avec les importance sup ou égale, 
    #d'abord pour le cpt permanent, ensuite pour les autres, avec prise necompte des lignes deja_traitees
    #list des id_ign des tronc_elem
    liste_id_ign_te=[a  for k, v in dico_ligne_pt.items() for a in v['lgn_trc_elem']]
    lignes_traitees=[]
    #appel des fonction de carac des lignes 
    df=at.import_donnes_base('local_otv','public', 'traf2015_bdt17_ed15_l','traf2015_bdt17_ed15_l_vertices_pgr')
    df2_chaussees=df.loc[df.nature.isin(['Autoroute', 'Quasi-autoroute', 'Route à 2 chaussées'])]
    df_avec_rd_pt,carac_rd_pt,lign_entrant_rdpt=at.identifier_rd_pt(df)
    df_lignes=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index
    #utilisret les fnctions de carac
    for k in sorted(dico_ligne_pt.keys(), key=lambda x: (dico_ligne_pt[x]['priorite'])): 
        ligne_comp=[x for x in at.lignes_troncon_elem(df_avec_rd_pt,carac_rd_pt, dico_ligne_pt[k]['ligne_base']) if x not in liste_id_ign_te+lignes_traitees]
        lignes_traitees+=ligne_comp
        dico_ligne_pt[k]['lignes_comp_base']=lgn_idcpt_lin.loc[lgn_idcpt_lin.id_ign.isin(ligne_comp)].id_ign.tolist()

    # mettre en forme et creation df de correspondance finale : 
    df_linearisee=pd.DataFrame(index=dico_ligne_pt.keys(), data=dico_ligne_pt.values())
    df_linearisee['lignes_comp_fin']=df_linearisee.apply(lambda x : x['lignes_comp_base'] + x['lgn_trc_elem'], axis=1 )
    dico_inverse=df_linearisee[['lignes_comp_fin']].to_dict()
    dico_inverse={a:k   for k, val  in dico_inverse['lignes_comp_fin'].items() for a in val}
    df_linearisee_fin=pd.DataFrame(index=dico_inverse.keys(), data=dico_inverse.values(), columns=['id_cpt_fin'])

    try : 
        df_linearisee_fin_glob=pd.concat([df_linearisee_fin_glob,df_linearisee_fin], axis=0, sort=False)
    except NameError : 
        df_linearisee_fin_glob=df_linearisee_fin.copy()

17-A10-423+0


In [278]:
#mise à jour avec 
df_linearisee_fin.to_csv(r'D:\temp\otv\test_linearisation\test_lin.csv')

In [337]:
ligne_pr_graph_transfert=ligne_pr_graph.copy()
ligne_pr_graph_transfert['geom']=ligne_pr_graph_transfert.apply(lambda x : dumps(x.geometry), axis=1)

In [340]:
ligne_pr_graph_transfert.loc[ligne_pr_graph_transfert['id_ign']=='TRONROUT0000000032954110']

,id_ign,id,nature,numero,importance,cl_admin,gestion,mise_serv,fictif,franchisst,...,id_tronc_e,nb_intrsct,src_geom,nb_intrs_1,tgt_geom,id_rdpt,nb_rte_rdp,nb_obj_sig,geometry,geom
2735,TRONROUT0000000032954110,119438,Autoroute,A10,1,Autoroute,ASF,NR,Non,NC,...,497.0,3,POINT(415106.9 6526825.29999999),2,POINT(415455.8 6528087.29999999),NaN,NaN,NaN,LINESTRING (415106.8999999997 6526825.29999999...,LINESTRING (415106.8999999996740371 6526825.29...
